# Data Exploration and Pipeline

The database contains `Users`, `Orders`, and `Providers` tables. 

Partners are the companies who sell surplus items on the marketplace.
A cohort consists of customers who made their first order within the same month (M0). 
M1 retention is the share of customers who have made at least one purchase one month after their first purchase month.

Explore the data with Sql to investigate the following:

- The top 10 partners by sales
- Customers’ favourite partner segments (default offer types). 
- What is the M1 retention for any given customer cohort. 

## Connect to the database 
The connection to the sqlite datbase is achieved through the `jupysql` python library. This allows querying the database from jupyter notebook. Alternative tools are SQLMagic

In [1]:
import sqlite3
con = sqlite3.connect("./data/mock_resq.db") 

%load_ext sql
%config SqlMagic.displaylimit = None

%sql sqlite:///data/mock_resq.db

displaylimit: Value None will be treated as 0 (no limit)

Connecting to 'sqlite:///data/mock_resq.db'

### How many user are in the database

In [2]:
%%sql
SELECT COUNT(*) AS USER_COUNT
FROM USERS

Running query in 'sqlite:///data/mock_resq.db'

USER_COUNT
358366


### How many countries do the users come from

In [3]:
%%sql
SELECT COUNT(DISTINCT COUNTRY) AS COUNTRY_COUNT
FROM USERS

Running query in 'sqlite:///data/mock_resq.db'

COUNTRY_COUNT
111


### Which top 10 countries have the most users

In [4]:
%%sql 
SELECT COUNTRY, COUNT(*) AS USER_COUNT
FROM USERS 
GROUP BY COUNTRY
ORDER BY USER_COUNT DESC
LIMIT 10

Running query in 'sqlite:///data/mock_resq.db'

country,USER_COUNT
FI,339573
SE,8961
EE,6505
DE,512
AX,434
ES,209
FR,200
GB,190
AT,182
NL,152


#### Inference

-  There are **358,366** users in the database who are from **111** different countries. 
-  The top 10 countries where the users come from are Finland (FI), Sweden (SE), Estonia(EE), Germany(DE), Åland (AX), Spain (ES), France (FR), Great Britain (GB), Austria (AT) and Netherlands (NL). 
-  Ninety-four perecent of users (94% i.e **339573** users) are from Finland, followed by Sweden which has **8961** users. 

### How many Providers are in the database

In [5]:
%%sql
SELECT COUNT(ID) AS PROVIDER_COUNT
FROM PROVIDERS

Running query in 'sqlite:///data/mock_resq.db'

PROVIDER_COUNT
4337


### Do providers have multiple offer types?

In [6]:
%%sql
SELECT ID AS PROVIDER, COUNT(DEFAULTOFFERTYPE) AS OFFER_TYPE_COUNT
FROM PROVIDERS
GROUP BY ID
ORDER BY OFFER_TYPE_COUNT DESC
LIMIT 5

Running query in 'sqlite:///data/mock_resq.db'

PROVIDER,OFFER_TYPE_COUNT
9222930112446389796,1
9217379655006460479,1
9215371507696178188,1
9214584721622525154,1
9212615296993900753,1


### How many providers per country

In [7]:
%%sql 
SELECT COUNTRY, COUNT(ID) AS PROVIDER_COUNT
FROM PROVIDERS
GROUP BY COUNTRY
ORDER BY PROVIDER_COUNT DESC

Running query in 'sqlite:///data/mock_resq.db'

country,PROVIDER_COUNT
fin,4095
est,154
swe,84
pol,2
deu,2


#### Inference

- There are **4337** providers in the database, with each provider having exactly one offer type (meal, snack, dessert, ingredients, flowers etc).  
- The providers are from Finland (FIN), Estonia (EST), Sweden (SWE), Poland (POL) and Germany (DEU). 
- Over **94 percent (4095)** of the providers are from Finland, followed by Estonia with **154** providers. 
- Poland and Germany has two (2) providers each. 


`Questions:` 

Are **partners** the same as **providers** ? If yes, Why the difference in numenclature in Db and instructions? 

The **Users** table has the two-letter [ISO-3166 country codes](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes) (example FI, SE, ES), while the **Providers** table uses the three-letter country code (example FIN, SWE, POL). Is there a perculiar reason for this ? 


All Providers have exactly one offer type in the database, however the numenclature of the offer says **defaultOfferType** and **partner segment**. Do providers have multiple offer types ? Could the numenclature be harmonised ?


## `Now answering the Analyst's questions`

## Top 10 partners by sales

In [8]:
%%sql 
SELECT PROVIDERID, SUM(SALES) AS TOTAL_SALES
FROM ORDERS
GROUP BY PROVIDERID
ORDER BY TOTAL_SALES DESC
LIMIT 10

Running query in 'sqlite:///data/mock_resq.db'

providerId,TOTAL_SALES
7198110370745783236,10917800
8312310143652755348,7467750
8097235958083241788,2383700
3865474760205653333,2223400
8084884958338058541,1868140
4734853230275691017,1702100
5305286819167536850,1690500
1066258454353124935,1568100
7642201963087705313,1472000
4014236829817167297,1457000


How many currencies are there in the database?

In [9]:
%%sql 
SELECT DISTINCT CURRENCY
FROM ORDERS

Running query in 'sqlite:///data/mock_resq.db'

currency
eur
sek


There are only **Euro(EUR)** and **Swedish Krone (SEK)** currencies in the database. 

`Question:` Is the conversion rate to the base currency, most likely in Euro, saved during the order payment? This will be useful in transforming the sales figures into a commmon currency for consistent reporting. 

How about **top 10 partners by sales in the respective sales currencies** ?

In [10]:
%%sql 
SELECT PROVIDERID, CURRENCY, SUM(SALES) AS TOTAL_SALES
FROM ORDERS
GROUP BY PROVIDERID, CURRENCY
ORDER BY TOTAL_SALES DESC
LIMIT 10

Running query in 'sqlite:///data/mock_resq.db'

providerId,currency,TOTAL_SALES
7198110370745783236,sek,10917800
8312310143652755348,sek,7467750
8097235958083241788,sek,2383700
3865474760205653333,sek,2223400
8084884958338058541,eur,1868140
4734853230275691017,sek,1702100
5305286819167536850,sek,1690500
1066258454353124935,sek,1568100
7642201963087705313,sek,1472000
4014236829817167297,sek,1457000


The same providers are in the top 10

## Customers’ favourite partner segments (default offer types).

In [11]:
%%sql 
SELECT P.DEFAULTOFFERTYPE AS PARTNER_SEGMENT, SUM(O.QUANTITY) AS SUM_ORDER_QUANTITY
FROM ORDERS O
JOIN PROVIDERS P ON O.PROVIDERID = P.ID
GROUP BY P.DEFAULTOFFERTYPE
ORDER BY SUM_ORDER_QUANTITY DESC
LIMIT 1

Running query in 'sqlite:///data/mock_resq.db'

PARTNER_SEGMENT,SUM_ORDER_QUANTITY
meal,305254


The Customers' favourite partner segment is **meal** with **305254** orders. 

## What is the M1 retention for any given customer cohort. 

Checking table definition and order table column types

In [12]:
%sql SELECT * FROM SQLITE_MASTER where TYPE='table'

Running query in 'sqlite:///data/mock_resq.db'

type,name,tbl_name,rootpage,sql
table,orders,orders,2,"CREATE TABLE orders (id, createdAt, userId, quantity, refunded, currency, sales, providerId)"
table,providers,providers,4420,"CREATE TABLE providers (id, defaultOfferType, country, registeredDate)"
table,users,users,4465,"CREATE TABLE users (id, country, registeredDate)"


In [13]:
%sql SELECT NAME, TYPE FROM PRAGMA_TABLE_INFO('ORDERS')

Running query in 'sqlite:///data/mock_resq.db'

name,type
id,
createdAt,
userId,
quantity,
refunded,
currency,
sales,
providerId,


No column types ? This is quit strange. The Date fields needs to be converted for date and aggregation functions

Now creating cohorts and computing the M1 retention share
A cohort consists of customers who made their first order within the same month (M0). 
M1 retention is the share of customers who have made at least one purchase one month after their first purchase month

In [14]:
%%sql
WITH FIRSTORDERDATES AS (
    SELECT 
        USERID, 
        MIN(DATE(CREATEDAT)) AS FIRST_ORDER_DATE
    FROM ORDERS
    GROUP BY USERID
),
CUSTOMERCOUNT AS (
    SELECT COUNT(*) AS ALLCUSTOMERS FROM USERS
),
M1RETENTIONCUSTOMERS AS (
    SELECT USERID, MAX(COHORT) AS COHORT
    FROM (
            SELECT 
            O.USERID, 
            ((strftime('%Y', O.CREATEDAT) - strftime('%Y', FOD.FIRST_ORDER_DATE)) * 12) + 
            (strftime('%m', O.CREATEDAT) - strftime('%m', FOD.FIRST_ORDER_DATE)) AS COHORT
        FROM ORDERS O
        LEFT JOIN FIRSTORDERDATES FOD ON O.USERID = FOD.USERID
    ) AS HISTORICALCOHORTS
    WHERE COHORT>0 
    GROUP BY USERID  
)


SELECT 
    (CAST((SELECT COUNT(*) FROM M1RETENTIONCUSTOMERS) AS REAL) / 
    CAST((SELECT AllCustomers FROM CustomerCount) AS REAL)) * 100 AS M1_SHARE_PERCENTAGE

Running query in 'sqlite:///data/mock_resq.db'

M1_SHARE_PERCENTAGE
14.386688469330238


`Answer` The M1 retention rate is **14.39** percent. This means that **14.39** percent of customers (**51,557** customers) made at least one purchase one month after their first purchase month.

## Building the data pipeline

First install **dbt-core** and **dbt-sqlite** with `pip install --quiet dbt-core dbt-sqlite`.

[dbt-sqlite](https://github.com/codeforkjeff/dbt-sqlite) version must align with that of dbt-core for the setup to work properly

If the project is setup with the exported virtual enviromnet file **environment.yml**, there is no need to reinstall these dependencies. Check that they are installed with `pip show  dbt-core dbt-sqlite`

The dbt init command was used to initialize the dbt project is in subdirectory **ResqDbPipeline**. Therefore all dbt commands should be run in that subdirectory. 

### Connecting profile for Sqlite database

In [15]:
import os

dbt_project = "./ResqDbPipeline"
os.makedirs(dbt_project, exist_ok=True)

In [16]:
%%writefile {dbt_project}/profiles.yml
ResqDbPipeline:
  target: dev
  outputs:
    dev:
      type: sqlite
      threads: 1
      database: 'database'
      schema: 'main'
      schemas_and_paths:
        main: '../data/mock_resq.db'
      schema_directory: '../data'

Overwriting ./ResqDbPipeline/profiles.yml


### Testing sqlite database connection

In [17]:
!cd {dbt_project} && dbt debug

14:08:18  Running with dbt=1.5.0
14:08:18  dbt version: 1.5.0
14:08:18  python version: 3.12.6
14:08:18  python path: /opt/anaconda3/envs/resq/bin/python
14:08:18  os info: macOS-14.6-arm64-arm-64bit
14:08:18  Using profiles.yml file at /Users/asare/source/ResqCLV/ResqDbPipeline/profiles.yml
14:08:18  Using dbt_project.yml file at /Users/asare/source/ResqCLV/ResqDbPipeline/dbt_project.yml
14:08:18  Configuration:
14:08:18    profiles.yml file [OK found and valid]
14:08:18    dbt_project.yml file [OK found and valid]
14:08:18  Required dependencies:
14:08:18   - git [OK found]

14:08:18  Connection:
14:08:18    database: database
14:08:18    schema: main
14:08:18    schemas_and_paths: {'main': '../data/mock_resq.db'}
14:08:18    schema_directory: ../data
14:08:18    Connection test: [OK connection ok]

14:08:18  All checks passed!


In [18]:
!cd {dbt_project} && dbt run --full-refresh

14:08:20  Running with dbt=1.5.0
14:08:20  Unable to do partial parsing because saved manifest not found. Starting full parse.
14:08:20  Encountered an error:
Compilation Error
  dbt found two macros named "materialization_table_default" in the project
  "dbt".
   To fix this error, rename or remove one of the following macros:
      - macros/materializations/models/table/table.sql
      - macros/materializations/models/table.sql


This error seem strange. This seems like a bug similar errors reported here: 
- https://discourse.getdbt.com/t/dbt-compilation-error-materialization-table-default/16375

- https://discourse.getdbt.com/t/materialization-table-default-macro-occurs/14973/2

### Next Action
Proceeding with good old python scripts since this is a a **`simple pipeline`**. There is only one data source which is already in the sqlite database, so there is no extraction step, but there will be a step that tranforms the data with queries and joins, and then create Views. 

### Top partners view script

In [19]:
%%writefile data/sql/top_partners.sql
DROP VIEW IF EXISTS TOP_PARTNERS;

CREATE VIEW TOP_PARTNERS 
AS 
    SELECT 
        PROVIDERID, 
        CURRENCY,
        SUM(SALES) AS TOTAL_SALES
    FROM ORDERS
    GROUP BY PROVIDERID, CURRENCY
    ORDER BY TOTAL_SALES DESC;

Overwriting data/sql/top_partners.sql


### Parter segment order quantity view script

In [20]:
%%writefile data/sql/partner_segment_order_quantity.sql

DROP VIEW IF EXISTS PARTNET_SEGMENT_ORDER_QUANTIY;

CREATE VIEW PARTNET_SEGMENT_ORDER_QUANTIY
AS 
    SELECT 
        P.DEFAULTOFFERTYPE AS PARTNER_SEGMENT, 
        SUM(O.QUANTITY) AS SUM_ORDER_QUANTITY
    FROM ORDERS O
    JOIN PROVIDERS P ON O.PROVIDERID = P.ID
    GROUP BY P.DEFAULTOFFERTYPE
    ORDER BY SUM_ORDER_QUANTITY DESC;

Overwriting data/sql/partner_segment_order_quantity.sql


### Customer cohort view

In [21]:
%%writefile data/sql/customer_cohort.sql
DROP VIEW IF EXISTS CUSTOMER_COHORT;

CREATE VIEW CUSTOMER_COHORT
AS
    WITH FIRSTORDERDATES AS (
        SELECT 
            USERID, 
            MIN(DATE(CREATEDAT)) AS FIRST_ORDER_DATE
        FROM ORDERS
        GROUP BY USERID
    )
    
    SELECT 
        USERID, 
        MAX(COHORT) AS COHORT
    FROM (
            SELECT 
                O.USERID, 
                ((strftime('%Y', O.CREATEDAT) - strftime('%Y', FOD.FIRST_ORDER_DATE)) * 12) + 
                (strftime('%m', O.CREATEDAT) - strftime('%m', FOD.FIRST_ORDER_DATE)) AS COHORT
            FROM ORDERS O
            LEFT JOIN FIRSTORDERDATES FOD ON O.USERID = FOD.USERID
    ) AS HISTORICALCOHORTS
    GROUP BY USERID  
    ORDER BY COHORT DESC;

Overwriting data/sql/customer_cohort.sql


In [22]:
import sqlite3
import pandas as pd
from datetime import datetime


class DataPipeline(object):
    db_path = "./data/mock_resq.db"

    def __init__(self):
        super().__init__()
        self._log("Initializing pipeline")
        self._create_connection()
        self._create_customer_cohorts_view()
        self._create_top_partners_by_sales_view()
        self._create_partner_segment_order_quantity_view()
        self._close_connection()
        self._log("Done")

    def top_n_partners(self, top_n):
        """
        Return top n partners by sales
        """
        
        sql = "SELECT * FROM TOP_PARTNERS LIMIT ?"
        return self._execute_query(query= sql, param=(top_n,))

    def customers_top_partner_segment(self, top_n=None):
        """
        Returns the order quantity of the top n partner segment.
        If top_n is not specified, it returns the favourite partner segment
        """

        top_n = 1 if top_n is None else top_n
        sql = "SELECT * FROM PARTNET_SEGMENT_ORDER_QUANTIY LIMIT ?"
        
        return self._execute_query(query=sql, param=(top_n,))

    def m1_retention_rate(self):
        """
        Returns M1 retention rate
        """

        sql = """
                SELECT 
                    (
                        CAST((SELECT COUNT(*) FROM CUSTOMER_COHORT WHERE COHORT > 0) AS REAL) / 
                        CAST((SELECT COUNT(*)  FROM USERS) AS REAL)
                    ) * 100 AS M1_SHARE_PERCENTAGE
              """
        return self._execute_query(query=sql)

    def _create_top_partners_by_sales_view(self):
        """
        Creates top partners by sales view
        """

        self._log("Creating top partners by sales view in database")
        script_path = "./data/sql/top_partners.sql"
        self._execute_script(script_path)

    def _create_partner_segment_order_quantity_view(self):
        """
        Creates partner segments and order quantity view 
        """

        self._log("Creating partner segments and order quantity view in database")
        script_path = "./data/sql/partner_segment_order_quantity.sql"
        self._execute_script(script_path)

    def _create_customer_cohorts_view(self):
        """
        Creates customer cohorts view
        """

        self._log("Creating customer cohorts view in database")
        script_path = "./data/sql/customer_cohort.sql"
        self._execute_script(script_path)

    def _create_connection(self):
        """
        Creates a database connection
        """

        self.conn = sqlite3.connect(self.db_path)

    def _close_connection(self):
        """
        Closes existing database connection.
        """

        if self.conn:
            self.conn.commit()
            self.conn.close()

    def _execute_script(self, script_path):
        """
        Execute SQL script from file 
        """

        with open(script_path, "r") as file:
            sql_script = file.read()

        cursor = self.conn.cursor()
        cursor.executescript(sql_script)

    def _execute_query(self, query, param=None):
        """
        Execute SQL query with parameters 
        """

        self._create_connection()
        with self.conn as connection:
            return pd.read_sql_query(sql=query, con=connection, params=param)
    
    def _log(self, text):
        print(datetime.now().strftime('%H:%M:%S'), text)

In [23]:
pipeline = DataPipeline()

17:08:21 Initializing pipeline
17:08:21 Creating customer cohorts view in database
17:08:21 Creating top partners by sales view in database
17:08:21 Creating partner segments and order quantity view in database
17:08:21 Done


The Analyst can now use the pipeline to create presentation tables, as shown below


Favourite partner segment:

In [24]:
pipeline.customers_top_partner_segment()

,PARTNER_SEGMENT,SUM_ORDER_QUANTITY
0,meal,305254


Top 3 favourite partner segment:

In [25]:
pipeline.customers_top_partner_segment(top_n = 3)

,PARTNER_SEGMENT,SUM_ORDER_QUANTITY
0,meal,305254
1,snack,63182
2,grocery-bag,29884


Top N partners by sales:

In [26]:
pipeline.top_n_partners(top_n=10)

,providerId,currency,TOTAL_SALES
0,7198110370745783236,sek,10917800
1,8312310143652755348,sek,7467750
2,8097235958083241788,sek,2383700
3,3865474760205653333,sek,2223400
4,8084884958338058541,eur,1868140
5,4734853230275691017,sek,1702100
6,5305286819167536850,sek,1690500
7,1066258454353124935,sek,1568100
8,7642201963087705313,sek,1472000
9,4014236829817167297,sek,1457000


M1 retention rate:

In [27]:
pipeline.m1_retention_rate()

,M1_SHARE_PERCENTAGE
0,14.386688
